In [3]:
#importing and creating pipelines
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
#standard syntax
pipe_long = Pipeline([("scaler",MinMaxScaler()),("svm",SVC())])
#abbreviated syntax
pipe_short = make_pipeline(MinMaxScaler(),SVC())

In [4]:
pipe_short.steps

[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('svc',
  SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
      max_iter=-1, probability=False, random_state=None, shrinking=True,
      tol=0.001, verbose=False))]

In [6]:
from sklearn.preprocessing import Normalizer
pipe = make_pipeline(MinMaxScaler(),Normalizer(),MinMaxScaler())
pipe.steps

[('minmaxscaler-1', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('normalizer', Normalizer(copy=True, norm='l2')),
 ('minmaxscaler-2', MinMaxScaler(copy=True, feature_range=(0, 1)))]

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer = load_breast_cancer()
X_train,X_test,y_train,y_test = train_test_split(cancer.data,cancer.target,random_state=0)
pipe = make_pipeline(MinMaxScaler(),SVC())
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)

0.972027972027972

In [8]:
#tuning the parameters 
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C':[0.01,0.1,1,10,100],'svc__gamma':[0.01,0.1,1,10,100]}
grid = GridSearchCV(pipe,param_grid=param_grid, cv=5)
grid.fit(X_train,y_train)
print("best cross-validation accuracy: ",grid.best_score_)
print("test set score: ",grid.score(X_test,y_test))
print("best parameters: ",grid.best_params_)

best cross-validation accuracy:  0.9812311901504789
test set score:  0.972027972027972
best parameters:  {'svc__C': 1, 'svc__gamma': 1}


In [9]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('svc',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma=1, kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [10]:
grid.best_estimator_.named_steps

{'minmaxscaler': MinMaxScaler(copy=True, feature_range=(0, 1)),
 'svc': SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
     probability=False, random_state=None, shrinking=True, tol=0.001,
     verbose=False)}

In [11]:
grid.best_estimator_.named_steps["svc"]

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [12]:
grid.best_estimator_.named_steps["svc"].n_support_

array([45, 40], dtype=int32)

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
iris = load_iris()
svm = SVC()
cross_val_score(svm,iris.data,iris.target)


array([0.96666667, 0.96666667, 0.96666667, 0.93333333, 1.        ])

In [14]:
from sklearn.model_selection import KFold
kf = KFold(shuffle=True,random_state=0)
kf

KFold(n_splits=5, random_state=0, shuffle=True)

In [16]:
kf.get_n_splits()

5

In [18]:
for rest_index,fold_index in kf.split(iris.data):
    print('current fold: ',fold_index)
    print("the rest of the training set: ",rest_index)
    X_rest,X_fold = iris.data[rest_index],iris.data[fold_index]
    y_rest,y_fold = iris.target[rest_index],iris.target[fold_index]
    svm.fit(X_rest,y_rest)
    #print(svm.score(X_fold,y_fold))

current fold:  [  7   8  16  22  24  26  33  37  40  44  45  51  54  62  63  66  71  73
  76  78  86  90  93  97 100 107 114 121 126 134]
the rest of the training set:  [  0   1   2   3   4   5   6   9  10  11  12  13  14  15  17  18  19  20
  21  23  25  27  28  29  30  31  32  34  35  36  38  39  41  42  43  46
  47  48  49  50  52  53  55  56  57  58  59  60  61  64  65  67  68  69
  70  72  74  75  77  79  80  81  82  83  84  85  87  88  89  91  92  94
  95  96  98  99 101 102 103 104 105 106 108 109 110 111 112 113 115 116
 117 118 119 120 122 123 124 125 127 128 129 130 131 132 133 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
current fold:  [  2  10  18  27  43  50  56  59  60  61  69  80  83  84  92 106 108 112
 116 119 123 127 132 133 135 137 141 144 146 147]
the rest of the training set:  [  0   1   3   4   5   6   7   8   9  11  12  13  14  15  16  17  19  20
  21  22  23  24  25  26  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  44  45  46  47

1.0
0.8
1.0
0.9666666666666667
0.9333333333333333


In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,train_size=6,test_size=None,random_state=0)
svm = SVC()
svm.fit(X_train,y_train)
svm.decision_function(X_test[0:2])

array([[ 0.87798384, -0.16723586,  2.20434741],
       [ 1.02135693, -0.15160776,  2.14456263]])

In [20]:
y_test[0:2]

array([2, 1])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=0)
import math
import numpy as np
def dist(x1,x2):
    return np.linalg.norm(x1-x2)
n_train = X_train.shape[0]
n_test = X_test.shape[0]
dist_own = math.inf*np.ones(n_train)
dist_other = math.inf*np.ones(n_train)
for i in range(n_train-1):
    for j in range(i+1,n_train):
        current_dist = dist(X_train[i],X_train[j])
        if (y_train[i]==y_train[j]):
            if (current_dist < dist_own[i]):
                dist_own[i] = current_dist
            if (current_dist < dist_own[j]):
                dist_own[j] - current_dist
        else:
            if (current_dist < dist_other[i]):
                dist_other[i] = current_dist
            if (current_dist < dist_other[j]):
                dist
            